# Phonation onset study

## Main results

### Global configuration + function definitions

In [ ]:
from tqdm.notebook import tqdm
from os.path import isfile, splitext
import itertools, functools
from pprint import pprint
from IPython.core.debugger import set_trace

import h5py
import numpy as np
import jax
from jax import numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import dolfin as dfn

from femvf import load, statefile as sf, meshutils
from blockarray import h5utils as bh5utils, blockvec as bv, linalg as bla
from vfsig import modal
from vfsig.misc import resample_over_uniform_period
from femvf.meshutils import process_meshlabel_to_dofs, verts_from_mesh_func

import libhopf
import libsetup
import libsignal
import h5utils
from postprocutils import postprocess

from libsetup import load_transient_model, load_hopf_model

In [ ]:
from main_onsetpressure import (
    ExpParamBasic, ExpParamFreqPenalty, 
    setup_dyna_model, setup_parameterization
)
from exputils import iter_parameters

In [ ]:
## Set default experiment parameters
CLSCALE = 0.5

# EMODS = np.arange(2.5, 20, 2.5) * 1e3 * 10
EMODS = 2.0 * np.arange(1, 10, 2) * 1e3 * 10
default_params_basic_dict = {
    'MeshName': f'M5_CB_GA3_CL{CLSCALE:.2f}',
    'Ecov': 2.0e4,
    'Ebod': 2.0e4,
    'ParamOption': 'const_shape',
    'Functional': 'OnsetPressure',
    'H': 1e-3,
    'EigTarget': 'LARGEST_MAGNITUDE',
    'SepPoint': 'separation-inf'
} 
default_params_penalty_dict = {
    'MeshName': f'M5_CB_GA3_CL{CLSCALE:.2f}',
    'Ecov': 2.0e4,
    'Ebod': 2.0e4,
    'ParamOption': 'all',
    'Functional': {
        'Name': 'OnsetPressure',
        'omega': -1,
        'beta': 1000
    },
    'H': 1e-3
} 

DEFAULT_PARAMS_BASIC = ExpParamBasic(default_params_basic_dict)
DEFAULT_PARAMS_PENALTY = ExpParamFreqPenalty(default_params_penalty_dict)

In [ ]:
## Functions for loading sensitivity/minimization simulation results

def load_sensitivity_scalars(
        substitute_params,
        dset_name, 
        default_params=DEFAULT_PARAMS_BASIC,
        load_dir='out/sensitivity'
    ):
    """
    Return `BlockVector` instance for the given parameter set
    """
    params = default_params.substitute(substitute_params)
    fpath = f'{load_dir}/{params.to_str()}.h5'
    with h5py.File(fpath, mode='r') as f:
        return f[dset_name][:]

def load_sensitivity_vectors(
        substitute_params,
        group_name, 
        default_params=DEFAULT_PARAMS_BASIC,
        load_dir='out/sensitivity'
    ):
    """
    Return a list of properties `BlockVector` instances over given parameters
    """
    params = default_params.substitute(substitute_params)
    fpath = f'{load_dir}/{params.to_str()}.h5'
    with h5py.File(fpath, mode='r') as f:
        dset_name = list(f[group_name].keys())[0]
        N = f[group_name][dset_name].shape[0]
        prop = [
            bh5utils.read_block_vector_from_group(f[group_name], nvec=n)
            for n in range(N)
        ]
        return tuple(prop)

def load_minimization_vectors(
        substitute_params,
        group_name, 
        it=-1,
        default_params=DEFAULT_PARAMS_PENALTY,
        load_dir='out/minimization'
    ):
    """
    Return a list of properties `BlockVector` instances over given parameters
    """
    prop = []
    for params in iter_parameters(substitute_params, default_params):
        fpath = f'{load_dir}/{params.to_str()}.h5'
        with h5py.File(fpath, mode='r') as f:
            prop.append(bh5utils.read_block_vector_from_group(f[group_name], nvec=it))
    return prop

In [ ]:
def form_basis(basis_vectors):
    """
    Return a matrix with columns from basis vectors
    """
    return np.stack(basis_vectors, axis=-1)
    
def form_principal_sensitivity(eigvals):
    """
    Return a matrix with eigenvalues on the diagonal
    """
    return np.diag(eigvals)

def eig_argsort(eigvals, axis=-1, sort_by='abs'):
    """
    Sort a set of eigenvalues (and associated vectors) in ascending order
    """
    kwargs = {'axis': axis}
    if sort_by == 'abs':
        idx_sort = np.argsort(np.abs(eigvals), **kwargs)
    elif sort_by == 'real':
        idx_sort = np.argsort(np.real(eigvals), **kwargs)
    elif sort_by == 'imag':
        idx_sort = np.argsort(np.imag(eigvals), **kwargs)
    else:
        raise ValueError("Unknown sorting type")
        
    return idx_sort

def eig_sort(eigvals, *args, axis=-1, sort_by='abs'):
    """
    Sort a set of eigenvalues (and associated vectors) in ascending order
    """
    kwargs = {'axis': axis}
    idx_sort = eig_argsort(eigvals, **kwargs, sort_by=sort_by)
        
    if len(args) == 0:
        return np.take_along_axis(eigvals, idx_sort, **kwargs)
    else:
        return (
            (np.take_along_axis(eigvals, idx_sort, **kwargs),) 
            + tuple(np.take_along_axis(arg, idx_sort, **kwargs) for arg in args)
        )

def form_projector(Z, A=np.array(1.0)):
    """
    Form the projecter onto a subspace
    
    Parameters
    ----------
    Z : ArrayLike
        The basis spanning the subspace
    A : ArrayLike
        The matrix representing the inner product used to define orthogonality.
        Orthogonality of vectors `u` and `v` w.r.t. `A` implies
            `u.T @ A @ v == 0`
    """
    # The A-orthogonal projector onto the subspace spanned by `Z` 
    # (basis vectors in columns) is given by:
    # See ("Numerical Linear Algebra", Trefethen and Bau, 1997)
    return Z @ np.linalg.inv(np.dot(Z.T, A).dot(Z)) @ np.dot(Z.T, A)

# def project_eig(eigvals, eigvecs, basis, inner=np.array(1.0)):
#     """
#     Project an eigendecomposition into a subspace
#     """
#     LMBDA = np.diag(aa)
#     Z = eigvecs
#     A = inner
#     pass
#     # np.dot(np.dot(PROJ.T, Z.T), ZEIG) @ LMBDA @ np.dot(np.dot(ZEIG.T, Z), PROJ)

In [ ]:
HOPF_MODEL, *_ = setup_dyna_model(DEFAULT_PARAMS_BASIC)
MESH = HOPF_MODEL.res.solid.forms['mesh.mesh']
VEC_CG1_DOFMAP = HOPF_MODEL.res.solid.forms['fspace.vector'].dofmap()
CG1_DOFMAP = HOPF_MODEL.res.solid.forms['fspace.scalar'].dofmap()
DG0_DOFMAP = HOPF_MODEL.res.solid.forms['fspace.scalar_dg0'].dofmap()
CELL_TO_SDOF_CG1 = CG1_DOFMAP.entity_closure_dofs(MESH, 2)
CELL_TO_SDOF = DG0_DOFMAP.entity_closure_dofs(MESH, 2)

In [ ]:
_mesh_names = (
    [f'M5_CB_GA3_CL{clscale:.2f}' for clscale in (0.5, 0.25, 0.125)]
    + [f'M5_CB_GA3_CL{clscale:.2f}_split' for clscale in (0.5, 0.25, 0.125)]
)

# This loads all Hopf models across different meshes
HOPF_MODELS = {
    mesh_name: setup_dyna_model(
        DEFAULT_PARAMS_BASIC.substitute({'MeshName': mesh_name})
    )[0]
    for mesh_name in _mesh_names
}

VDG0S = {
    mesh_name: dfn.FunctionSpace(model.res.solid.forms['mesh.mesh'], 'DG', 0)
    for mesh_name, model in HOPF_MODELS.items()
}
MEASURES = {
    mesh_name: dfn.Measure('dx', model.res.solid.forms['mesh.mesh'])
    for mesh_name, model in HOPF_MODELS.items()
}
MS_DG0 = {
    mesh_name: dfn.assemble(
        dfn.TrialFunction(v)*dfn.TestFunction(v)*MEASURES[mesh_name],
        tensor=dfn.PETScMatrix()
    ).mat()
    for mesh_name, v in VDG0S.items()
}

In [ ]:
# Get a list of vertices on the medial region
forms = HOPF_MODEL.res.solid.forms

cell_func = forms['mesh.cell_function']
cell_label_to_id = forms['mesh.cell_label_to_id']
facet_func = forms['mesh.facet_function']
facet_label_to_id = forms['mesh.facet_label_to_id']
print(facet_label_to_id, cell_label_to_id)
VERTS_MED = verts_from_mesh_func(MESH, facet_func, 3)
VERTS_DIR = verts_from_mesh_func(MESH, facet_func, 4)

VERT_TO_VDOF = dfn.vertex_to_dof_map(HOPF_MODEL.res.solid.forms['fspace.vector'])
VERT_TO_SDOF = dfn.vertex_to_dof_map(HOPF_MODEL.res.solid.forms['fspace.scalar'])

SDOF_TO_VERT = dfn.dof_to_vertex_map(HOPF_MODEL.res.solid.forms['fspace.scalar'])

FSPACE_CG1 = dfn.FunctionSpace(forms['mesh.mesh'], 'CG', 1)
FSPACE_DG0 = dfn.FunctionSpace(forms['mesh.mesh'], 'DG', 0)
V_DG0 = dfn.Function(FSPACE_DG0)
def project_DG0_to_CG1(f):
    V_DG0.vector()[:] = f
    return dfn.project(V_DG0, V=FSPACE_CG1).vector()[:]

### Debug: Check H5 file contents

In [ ]:
params = DEFAULT_PARAMS_BASIC
fpath = f'out/sensitivity/{params.to_str()}.h5'
with h5py.File(fpath, mode='r') as f:
    print(fpath)
    print(f.keys())
    print(f['grad_param/emod'])

### Plot configuration

In [ ]:
INCH = 1/2.54

FIG_STYLE = 'manuscript'

if FIG_STYLE == 'presentation':
    RCPARAMS = {
        'font.family': 'sans-serif',
        'font.size': 24
    }
    
    FIG_LX = 14.7 * INCH
    FIG_LX_WIDE = 30 * INCH
    FIG_LY = 13 * INCH
    FIG_LY_MAX = FIG_LY
    
    FIG_DIR = f"fig/presentation"
elif FIG_STYLE == 'manuscript':
    RCPARAMS = {
        'font.family': 'sans-serif',
        'font.size': 9
    }
    
    FIG_LX = 8.4 * INCH
    FIG_LX_WIDE = 17.4 * INCH
    FIG_LY = 10 * INCH
    FIG_LY_MAX = 23.4*INCH
    
    FIG_DIR = f"fig/manuscript"
else:
    raise ValueError("`FIG_STYLE` must be 'manuscript' or 'presentation'.")
    
if CLSCALE == 0.5:
    FIG_DIR_POSTFIX = ''
else:
    FIG_DIR_POSTFIX = f'{CLSCALE:.2f}'
    
if FIG_DIR_POSTFIX != '':
    FIG_DIR = f'{FIG_DIR}_{FIG_DIR_POSTFIX}'
    
mpl.rcParams.update(RCPARAMS)

FIG_EXT = 'pdf'

COLOR_CYCLE = mpl.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
## Pre-defined plotting routines

def plot_triplots(
        fig, axs, coordss, cellss, zs,
        plot_type='tripcolor',
        **plot_kwargs
    ):
    """
    Plot a set of `tri...` type plots with shared color limits
    
    The plots will use the same 'v' limits so that a shared colorbar can be 
    used.
    
    Parameters
    ----------
    fig : mpl.Figure
        The figure instance containing plots
    axs : List[mpl.Axes]
        A list of axes for each desired tri-type plot
    coords : List[np.ndarray]
        A list of mesh coordinates for each plot
    cells : List[np.ndarray]
         A list of cell connectivity information for each plot
    zs : List[np.ndarray]
        Value for color information on each plot
    
    Returns
    -------
    artists: List[mpl.artist.Artist]
        A list of artists created from each `tri*` plot
    """
    zmin = np.nanmin([np.nanmin(z) for z in zs])
    zmax = np.nanmax([np.nanmax(z) for z in zs])
    if 'norm' not in plot_kwargs:
        if 'vmin' not in plot_kwargs:
            plot_kwargs['vmin'] = zmin
        if 'vmax' not in plot_kwargs:
            plot_kwargs['vmax'] = zmax
        
    # print(zmin, zmax)
    
    if plot_type == 'tripcolor':
        artists = [
            ax.tripcolor(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    elif plot_type == 'tricontourf':
        artists = [
            ax.tricontourf(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    elif plot_type == 'tricontour':
        artists = [
            ax.tricontour(*coords.T, cells, z, **plot_kwargs)
            for ax, z, coords, cells in zip(axs, zs, coordss, cellss)
        ]
    else:
        raise ValueError(f"Unknown 'plot_type' '{plot_type}'")
    return artists

def grid_xy_axis_format(axs, xlabel, ylabel):
    """
    Format a set of axes so they have a shared x/y axis and label
    
    Parameters
    ----------
    axs : ArrayLike[n, m]
        2D array of axes objects in a grid
    """
    for ax in axs[:, 0]:
        ax.set_ylabel(ylabel)

    for ax in axs[-1, :]:
        ax.set_xlabel(xlabel)

    for ax in axs[:-1, :].flat:
        ax.xaxis.set_tick_params(labelbottom=False)
    for ax in axs[:, 1:].flat:
        ax.yaxis.set_tick_params(labelleft=False)

### Construct a layered/structured basis


In [ ]:
V = HOPF_MODEL.res.solid.forms['coeff.prop.emod'].function_space()
DX = dfn.Measure('dx', MESH)
xdofs = V.tabulate_dof_coordinates()[:, 0]
basis_functions = [dfn.Function(V) for n in range(4)]
basis_coeffvecs = [func.vector() for func in basis_functions]

_u = dfn.TrialFunction(V)
_du = dfn.TestFunction(V)
M = dfn.assemble(_u*_du*dfn.Measure('dx', MESH), tensor=dfn.PETScMatrix())

In [ ]:
cell_label_to_dofs = process_meshlabel_to_dofs(MESH, cell_func, cell_label_to_id, V.dofmap())
dofs_body = cell_label_to_dofs['body']
dofs_cover = cell_label_to_dofs['cover']

## Set variations in representing layered stiffness changes:
for vec in basis_coeffvecs:
    vec[:] = 0
# uniform
basis_coeffvecs[0][:] = 1

# increasing body - cover stiffness difference
basis_coeffvecs[1][dofs_body] = 1
basis_coeffvecs[1][dofs_cover] = -1

# increasing cover superior-to-inferior stiffness gradient (superior > inferior)
basis_coeffvecs[2][dofs_cover] = -1 * xdofs[dofs_cover]

# increasing body superior-to-inferior stiffness gradient (superior > inferior)
basis_coeffvecs[3][dofs_body] = -1 * xdofs[dofs_body]

## Orthonormalize the basis functions
def inner(x, y):
    return x.inner(M*y)

def orth_proj(x, y):
    y2norm = y.norm('l2')**2
    return inner(x, y)/inner(y, y) * y
        
def orthogonalize(x, *ys):
    res = x
    for y in ys:
        res = res - orth_proj(res, y)
    return res
    # return functools.reduce(lambda a, b: a - orth_proj(a, b), ys, x)

def normalize(x):
    return x/inner(x, x)**0.5

for n in range(1, len(basis_coeffvecs)):
    basis_coeffvecs[n] = orthogonalize(basis_coeffvecs[n], *basis_coeffvecs[:n])

for n in range(len(basis_coeffvecs)):
    basis_coeffvecs[n] = normalize(basis_coeffvecs[n])

In [ ]:
## Basis matrices

# Basis with variations in: uniform, body-cover
Z_BC = form_basis(basis_coeffvecs[:2])

# Basis with variations in: uniform, body-cover, inf-sup in cover
Z_BC_CINFSUP = form_basis(basis_coeffvecs[:3])

# Basis with variations in: uniform, body-cover, inf-sup in cover, and inf-sup in body
Z_BC_CINFSUP_BINFSUP = form_basis(basis_coeffvecs)

In [ ]:
## Test the basis is orthonormal
print(Z_BC_CINFSUP_BINFSUP.T @ M.mat()[:, :] @ Z_BC_CINFSUP_BINFSUP)

In [ ]:
# type(M.mat())

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY))

N = len(basis_coeffvecs)
gspec = mpl.gridspec.GridSpec(2, N, height_ratios=[0.025, 1])
ax_cbar = fig.add_subplot(gspec[0, :])
axs = np.array([fig.add_subplot(gspec[1, n]) for n in range(N)])

artists = plot_triplots(fig, axs, [MESH.coordinates()]*N, [MESH.cells()]*N, basis_coeffvecs, lw=0)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel("$E$ [kPa]")

for ax in axs:
    ax.set_aspect(1)

for ax in axs:
    ax.set_xlabel("$x$ [cm]")
    
descrs = [
    "uniform", "body-cover", "inf-sup cover", "inf-sup body"
]
eigvec_labels = [
    f"$\\vec{{ e }}_{{ \mathrm{{layer}}, {i+1} }}$:\n {descr}" 
    for i, descr in enumerate(descrs)
]
print(eigvec_labels)
for ax, eigvec_label in zip(axs, eigvec_labels):
    ax.text(0, 1, eigvec_label, transform=ax.transAxes, va='bottom')
    
axs[0].set_ylabel("$y$ [cm]")

for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)

fig.tight_layout()

fig.savefig(f"{FIG_DIR}/LayeredBasis.{FIG_EXT}")

### Conceptual sensitivity

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY*0.9), constrained_layout=True)
gs = gridspec.GridSpec(2, 4, height_ratios=[0.05, 1], figure=fig)
axs = np.array(
    [fig.add_subplot(gs[1, n], projection=None) for n in range(gs.ncols)]
)
ax_cbar = fig.add_subplot(gs[0, :])

# Sequence of principal curvature cases
As = [
    np.diag([10, 20]), np.diag([10, -5]), 
    np.diag([10, 0.001]), np.diag([10, 0.001])
]
Z = np.stack([[1, -1], [1, 1]], axis=-1)
Z = Z/np.diag(Z.T@Z)**0.5
print(Z.T@Z)
Hs = [Z @ A @ Z.T for A in As]
bs = [0]*3 + [10]

ec_0, eb_0 = 1, 1
ec_min, eb_min = 2, 5
ec = np.linspace(-5, 5, 100) + ec_min
eb = np.linspace(-8, 5, 50) + eb_min

# ec = np.linspace(-100, 10, 100) + ec_min
# eb = np.linspace(-10, 100, 50) + eb_min

EC, EB = np.meshgrid(ec-ec_min, eb-eb_min, indexing='ij')
E = np.stack([EC, EB], axis=-1)

pons = [
    np.sum((E @ H)*E, axis=-1) + b*E.dot(Z[:, 1])
    for H, b in zip(Hs, bs)
]
vmin = np.min([np.min(pon) for pon in pons])
vmax = np.max([np.max(pon) for pon in pons])

artists = [
    ax.contourf(eb, ec, pon, cmap=mpl.cm.viridis, vmin=vmin, vmax=vmax)
    for ax, pon in zip(axs, pons)
]

for ax, A in zip(axs, As):
    E_min = np.array([ec_min, eb_min])
    ax.arrow(*E_min[::-1], *Z[::-1, 0], width=0.05, fc='k')
    ax.arrow(*E_min[::-1], *Z[::-1, 1], width=0.05, fc='k')
    
    ax.annotate(f"$\\lambda={np.diag(A)[0]:.1f}$", (E_min+1.5*Z[:, 0])[::-1])
    ax.annotate(f"$\\lambda={np.diag(A)[1]:.1f}$", (E_min+1.5*Z[:, 1])[::-1])
    
    E_0 = np.array([ec_0, eb_0])
    dE = 0.05*Z@A@Z.T@(E_0-E_min)
    # print(E_0-E_min)
    ax.arrow(*E_0[::-1], *dE[::-1], width=0.05, fc='k')
    xy = (E_0+1.5*dE)[::-1]
    xytext = (E_0+0.8*1.5*dE+np.array([-1, 0]))[::-1]
    ax.annotate(f"$dp/dE$", xy, xytext=xytext)
    
    ax.annotate(f"$E_0$", E_0[::-1] + [0, 1])
    ax.annotate(f"$E_\mathrm{{min}}$", E_min[::-1] + [0, -1])

fig.colorbar(artists[-1], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel("Onset pressure [Pa]")

axs.flat[0].set_ylabel("$E_c$ [kPa]")
for ax, alph in zip(axs, 'ABCD'):
    ax.set_xlabel("$E_b$ [kPa]")
    ax.annotate(f"{alph}", (0.05, 1.05), xycoords=ax.transAxes, va='bottom')
    ax.set_aspect(1)
    
for ax in axs[1:]:
    ax.sharey(axs[0])
    ax.yaxis.set_tick_params(labelleft=False)
    
# fig.tight_layout()
fig.savefig(f"{FIG_DIR}/ConceptualSensitivity.{FIG_EXT}")

### Mesh and jacobian step size dependence

In [ ]:
emod_cov, emod_bod = (2e4, 6e4)
hs = np.array([1e-2, 1e-3, 1e-4, 1e-5])
hs = np.array([1e-3, 1e-4])
hs = np.array([1e-3])
clscales = np.array([0.5, 0.25, 0.125])
clscales = np.array([0.5, 0.25])
output_dir = 'out/independence'

# eig_target = 'LARGEST_REAL'
eig_target = 'LARGEST_MAGNITUDE'

NUM_EIG = 5
params = [
    DEFAULT_PARAMS_BASIC.substitute({
        'MeshName': f'M5_CB_GA3_CL{clscale:.2f}_split', 'H': h, 
        'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure',
        'EigTarget': eig_target
    })
    for h, clscale in itertools.product(hs, clscales)
]
models = [
    HOPF_MODELS[param['MeshName']] for param in params
]
Ms = [MS_DG0[param['MeshName']] for param in params]
cset_coords = [model.res.solid.forms['mesh.mesh'].coordinates() for model in models]
cset_cells = [model.res.solid.forms['mesh.mesh'].cells() for model in models]

cset_eigvecs = np.array([
    form_basis([
        evec['emod'][:] for evec in 
        load_sensitivity_vectors(
            param, 'hess_param', load_dir=output_dir
        )[:NUM_EIG]
    ])
    for param in params
], dtype=object).reshape(len(hs), len(clscales))

cset_eigvals = np.array([
    load_sensitivity_scalars(
        param, 'eigvals', load_dir=output_dir
    )[:NUM_EIG]
    for param in params
]).reshape(len(hs), len(clscales), NUM_EIG)

cset_eigvecs.shape

In [ ]:
print(cset_eigvals[0, 1])

In [ ]:
IDX_EIG = 0

In [ ]:
### Plot eigenvectors with varying step size and mesh size
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY_MAX), constrained_layout=True)
gs = gridspec.GridSpec(
    len(hs)+1, len(clscales), 
    figure=fig, height_ratios=[0.05]+len(hs)*[1]
)

axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(1, gs.nrows)],
    dtype=object
)
for ax in axs.flat:
    ax.set_aspect(1)
ax_cbar = fig.add_subplot(gs[0, :])


zs = [A[:, IDX_EIG] for A in cset_eigvecs.flat]
scales = [z.dot(M[:, :].dot(z))**0.5 for M, z in zip(Ms, zs)]
scales = [abs(z.max()-z.min()) for z in zs]
zs_norm = [z/scale for z, scale in zip(zs, scales)]

# Find/plot the location of the maximum for debugging
cell_max_idxs = [np.argmax(np.abs(z)) for z in zs] 
cell_max_coords = [
    np.sum(coords[cells[idx]], axis=0)/3 
    for coords, cells, idx in zip(cset_coords, cset_cells, cell_max_idxs)
]
# for ax, coords_max in zip(axs.flat, cell_max_coords):
#     ax.plot(*coords_max, marker='o', markersize=10)
    
artists = plot_triplots(
    fig, axs.flat, cset_coords, cset_cells, zs_norm
    # vmin=-0.001, vmax=0.001
)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax in axs[:, 0]:
    ax.set_ylabel("y [cm]")
    
for ax in axs[-1, :]:
    ax.set_xlabel("x [cm]")
    
for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)
for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax, h in zip(axs[:, -1], hs):
    ax.annotate(
        f"h: {h:.3e}", (1.05, 0.5), 
        rotation=-90, xycoords=ax.transAxes, va='center'
    )
    
for ax, clscale in zip(axs[0, :], clscales):
    ax.annotate(
        f"clscale: {clscale:.3f}", (0.5, 1.05), 
        rotation=0, xycoords=ax.transAxes, ha='center'
    )
    
fig.savefig(f'fig/manuscript/EigVecIndependence_Ecov{emod_cov:.2e}_Ebod{emod_bod:.2e}_{IDX_EIG:d}.{FIG_EXT}')

In [ ]:
fig, ax = plt.subplots(1, 1)

for h, eigvals_trend, eigvecs_trend in zip(hs, cset_eigvals, cset_eigvecs):
    es = [A[:, IDX_EIG] for A in eigvecs_trend]
    # es_scale = [np.abs(e).max() for e in es]
    es_scale = [e.dot(M[:, :].dot(e))**0.5 for M, e in zip(Ms, es)]
    es_normalized = [e/scale for e, scale in zip(es, es_scale)]
    # es_normalized = [e/(e.max()-e.min()) for e in es]
    scales = np.array([np.linalg.norm(e) for e in es_normalized])
    
    eigvals = eigvals_trend[:, IDX_EIG]

    print(eigvals, scales)
    print(eigvals*scales**2)
    
    ax.plot(
        1/clscales, eigvals*scales**2
    )
    
ax.set_xlabel("Mesh size refinement")
ax.set_ylabel(f"Eigenvalue {IDX_EIG+1:d}")

fig.tight_layout()
fig.savefig(f'fig/manuscript/EigValIndependence_Ecov{emod_cov:.2e}_Ebod{emod_bod:.2e}_{IDX_EIG:d}.{FIG_EXT}')

### Effect of moving separation point

In [ ]:
emod_cov, emod_bod = (2e4, 6e4)
clscale = 0.25
output_dir = 'out/separation_effect'

NUM_EIG = 5
params = [
    DEFAULT_PARAMS_BASIC.substitute({
        'MeshName': f'M5_CB_GA3_CL{clscale:.2f}_split',
        'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure',
        'EigTarget': 'LARGEST_MAGNITUDE', 'SepPoint': sep_point
    })
    for sep_point in ('separation-inf', 'separation-mid')
]
models = [
    HOPF_MODELS[param['MeshName']] for param in params
]
sep_values = [
    model.res.solid.forms['mesh.vertex_label_to_id'][param['SepPoint']]
    for model, param in zip(models, params)
]
sep_verts = [
    model.res.solid.forms['mesh.vertex_function'].where_equal(value)[0]
    for model, value in zip(models, sep_values)
]
sep_coords = [
    model.res.solid.forms['mesh.mesh'].coordinates()[idx]
    for idx, model in zip(sep_verts, models)
]

Ms = [MS_DG0[param['MeshName']] for param in params]
cset_coords = [model.res.solid.forms['mesh.mesh'].coordinates() for model in models]
cset_cells = [model.res.solid.forms['mesh.mesh'].cells() for model in models]

cset_eigvecs = [
    form_basis([
        evec['emod'][:] for evec in 
        load_sensitivity_vectors(
            param, 'hess_param', load_dir=output_dir
        )[:NUM_EIG]
    ])
    for param in params
]

cset_eigvals = np.array([
    load_sensitivity_scalars(
        param, 'eigvals', load_dir=output_dir
    )[:NUM_EIG]
    for param in params
])

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY_MAX), constrained_layout=True)
gs = gridspec.GridSpec(
    1+NUM_EIG, 2,
    figure=fig, height_ratios=[0.05]+[1]*NUM_EIG
)

axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(1, gs.nrows)],
    dtype=object
)
ax_cbar = fig.add_subplot(gs[0, :])

for ax in axs.flat:
    ax.set_aspect(1)

IDX = 0
zs = [evecs[:, ii] for ii, evecs in itertools.product(range(NUM_EIG), cset_eigvecs)]
scales = [abs(z.max()-z.min()) for z in zs]
# scales = [1]*len(zs)
zs_norm = [z/scale for z, scale in zip(zs, scales)]

artists = plot_triplots(
    fig, axs.flat, cset_coords*NUM_EIG, cset_cells*NUM_EIG, zs_norm
)

for axs_col, eigvals in zip(axs.T, cset_eigvals):
    for ax, eigval in zip(axs_col, eigvals):
        ax.annotate(f"$\\lambda = {eigval:.2f}$", (0.05, 0.95), xycoords=ax.transAxes, va='top')

for axs_col, sep_coord in zip(axs.T, sep_coords):
    for ax in axs_col:
        ax.plot(*sep_coords[0], marker='o', ms=5, mfc='k', mec='none')
        ax.plot(*sep_coords[1], marker='o', ms=5, mfc='k', mec='none')
        
        ax.plot(*sep_coord, marker='o', ms=10, mfc='none', mec='k')

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

grid_xy_axis_format(axs, "x [cm]", "y [cm]")

fig.savefig(f'{FIG_DIR}/SeparationEffect.{FIG_EXT}')

### Sensitivity results over varying linearization points

In [ ]:
## Load sensitivity information over a set of base parameter points
# `EMODS_COV` and `EMODS_BOD` are the base parameter points to load from
EMODS_COV = 1/4*2.0e3 * 10 * np.arange(1, 10, 2)
EMODS_BOD = 2.0e3 * 10 * np.arange(1, 10, 2)

EMODS_COV = 1/3*2.0e3 * 10 * np.arange(1, 10, 2)
EMODS_BOD = 2.0e3 * 10 * np.arange(1, 10, 2)

EMODS_COV = 1/2*2.0e3 * 10 * np.arange(1, 10, 2)
EMODS_BOD = 2.0e3 * 10 * np.arange(1, 10, 2)

EMODS_COV = 2.0e3 * 10 * np.arange(1, 10, 2)
EMODS_BOD = 2.0e3 * 10 * np.arange(1, 10, 2)

EMODS_COV = 1e4 * 10*np.array([1, 1/2, 1/3, 1/4])
EMODS_BOD = 1e4 * 10*np.array([1, 1, 1, 1])

EMODS_COV = 1e4 * 6*np.array([1, 1/3, 1/4])
EMODS_BOD = 1e4 * 6*np.array([1, 1, 1])


# functional_name = 'OnsetFrequency'
# functional_name = 'OnsetPressure'
# functional_name = 'OnsetPressureStrainEnergy'

# TODO: This could be used to generalize some plots
# functional_name_to_symb = {
#     'OnsetFrequency': 'f_\mathrm{onset}',
#     'OnsetPressure': 'p_\mathrm{onset}',
#     'OnsetPressureStrainEnergy': 'G'
# }
# functional_symb = functional_name_to_symb[functional_name]

cset_props = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'prop'
    )[0]
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_xhopf = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'state'
    )[0]
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

## Load first + second order sensitivities

# for onset frequency
# NOTE: The index `[0]` for gradient info is because the gradient is a single
# `BlockVector`
cset_grad_params_fon = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetFrequency'}, 
        'grad_param'
    )[0]
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_hess_params_fon_evecs = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetFrequency'}, 
        'hess_param'
    )
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_hess_params_fon_evals = [
    load_sensitivity_scalars(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetFrequency'}, 
        'eigvals'
    )
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

# for onset pressure
cset_grad_params_pon = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'grad_param'
    )[0]
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_hess_params_pon_evecs = [
    load_sensitivity_vectors(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'hess_param'
    )
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

cset_hess_params_pon_evals = [
    load_sensitivity_scalars(
        {'Ecov': emod_cov, 'Ebod': emod_bod, 'Functional': 'OnsetPressure'}, 
        'eigvals'
    )
    for emod_cov, emod_bod in zip(EMODS_COV, EMODS_BOD)
]

# emods_cov
cset_hess_params_pon_evals

#### 1st order sensitivity

In [ ]:
# %debug
## Plot 1st order sensitivity

# FUNCTIONAL_TYPE = 'OnsetFrequency'
FUNCTIONAL_TYPE = 'OnsetPressure'

BASIS_TYPE = 'Layered'
BASIS_TYPE = 'All'

idx_cases = slice(None, None) 
emods_cov = EMODS_COV[idx_cases]
emods_bod = EMODS_BOD[idx_cases]

if FUNCTIONAL_TYPE == 'OnsetPressure':
    params_lists = [cset_grad_params_pon[idx_cases]]
    symbols = [r"dp_\mathrm{on}/d{E}"]
elif FUNCTIONAL_TYPE == 'OnsetFrequency':
    params_lists = [cset_grad_params_fon[idx_cases]]
    symbols = [r"df_\mathrm{on}/d{E}"]
else:
    params_lists = [
        cset_grad_params_fon[idx_cases], 
        cset_grad_params_pon[idx_cases], 
        dppon[idx_cases]
    ]
    symbols = [
        r"df_\mathrm{on}/dE",
        r"dp_\mathrm{on}/dE",
        r"d\hat{p}_\mathrm{on}/dE"
    ]

N = len(symbols)
NUM_FUNCTIONAL_TYPE = len(params_lists[0])

if BASIS_TYPE == 'All':
    Z = np.ones((1, 1))
    Z = np.array(1)
    PROJ = np.array(1)
elif BASIS_TYPE == 'Layered':
    Z = Z_BC_CINFSUP_BINFSUP
    # A = 1.0
        
    PROJ = form_projector(Z, A=M.mat()[:, :])
    PROJ = form_projector(Z, A=1.0)
else:
    raise ValueError(f"Unknown `BASIS_TYPE` {BASIS_TYPE}")

fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True)
gs = mpl.gridspec.GridSpec(
    2*N, NUM_FUNCTIONAL_TYPE, figure=fig, height_ratios=[0.05, 0.95]*N
)

axs = np.array([
    [fig.add_subplot(gs[2*i+1, j]) for j in range(gs.ncols)] 
    for i in range(gs.nrows//2)
])
axs_cbar = np.array([fig.add_subplot(gs[2*i, :]) for i in range(gs.nrows//2)])

coords = MESH.coordinates()
cells = MESH.cells()

for ii, (grads, symb) in enumerate(zip(params_lists, symbols)):
    ## Plot emod sensitivity
    zs = [
        grad['emod'][:].dot(PROJ)[CELL_TO_SDOF] 
        for grad in grads
    ]
    
    if FUNCTIONAL_TYPE == 'OnsetPressure':
        kwargs = {'vmin': -6, 'vmax': 10}
    elif FUNCTIONAL_TYPE == 'OnsetFrequency':
        kwargs = {}
    
    artists = plot_triplots(
        fig, axs[ii, :], len(zs)*[coords], len(zs)*[cells], zs, 
        lw=0, **kwargs
    )
    
    fig.colorbar(artists[0], cax=axs_cbar[ii], orientation='horizontal')
    axs_cbar[ii].set_xlabel(f"${symb}$ [kPa]")
    # axs_cbar[ii].tick_params()

#     # Plot tmesh sensitivity
#     xy_med = coords[VERTS_MED]

#     med_dofs = np.array(
#         VEC_CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_MED)
#     ).reshape(-1, 2)
#     dir_dofs = np.array(
#         CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_DIR)
#     ).reshape(-1, 2)
#     for ax, dparams in zip(axs[ii, :], dparams_list):
#         tmesh = dparams['tmesh']
#         tmesh_med = tmesh[med_dofs]
#         ax.quiver(*xy_med.T, *tmesh_med.T, label="Shape traction")

for ax in axs.flat:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
    ax.set_ylim(0, 0.6)

case_labels = [
    f"({emod_cov/10/1e3:.1f} kPa, {emod_bod/10/1e3:.1f} kPa)" 
    for emod_cov, emod_bod in zip(emods_cov, emods_bod)
]
case_labels[0] = "$(E_{\mathrm{b}}, E_{\mathrm{c}})$ = \n" + case_labels[0]
for ax, case_label in zip(axs[0, :], case_labels):
    ax.text(0.05, 1.05, case_label, transform=ax.transAxes)
    
axs.flat[0].set_ylabel("y [cm]")

for ax in axs[:, 1:].flat: 
    ax.tick_params('y', labelleft=False)

# fig.tight_layout()
fig.savefig(f'{FIG_DIR}/Gradient_{FUNCTIONAL_TYPE}_{BASIS_TYPE}.{FIG_EXT}')

#### 2nd order sensitivity

In [ ]:
## Plot eigendecomposition of second order sensitivity

EIGVAL_ORDER = 'real'
BASIS_TYPE = 'All'

IDX_CASE = slice(None, None)
emods_cov = EMODS_COV[IDX_CASE]
emods_bod = EMODS_BOD[IDX_CASE]
if FUNCTIONAL_TYPE == 'OnsetPressure':
    eigvecs = cset_hess_params_pon_evecs[IDX_CASE]
    eigvals = cset_hess_params_pon_evals[IDX_CASE]
elif FUNCTIONAL_TYPE == 'OnsetFrequency':
    eigvecs = cset_hess_params_fon_evecs[IDX_CASE]
    eigvals = cset_hess_params_fon_evals[IDX_CASE]
ncase = len(emods_cov)

# Truncate the set of eigenvectors + values to have `neig` components for each 
# case
idx_eig = slice(None)
_eigvecs = [
    form_basis([evec['emod'][:] for evec in eigvecs[irow][idx_eig]]) 
    for irow in range(ncase)
]
_eigvals = [
    eigvals[irow][idx_eig]
    for irow in range(ncase)
]

# Compute eigendecompositions in a another basis
if BASIS_TYPE == 'All':
    # ZS = _eigvecs
    ZS = [np.array(1.0)]*ncase
    PROJ = np.array(1.0)
elif BASIS_TYPE == 'Layered':
    Z = Z_BC_CINFSUP_BINFSUP
    ZS = [Z_BC_CINFSUP_BINFSUP]*ncase
    PROJ = form_projector(Z, A=M.mat()[:, :])
    PROJ = form_projector(Z, A=1.0)
else:
    raise ValueError(f"Unknown `BASIS_TYPE` {BASIS_TYPE}")
    
# print(BASIS_TYPE)

subspace_eigdecomps = [
    np.linalg.eigh( 
        np.dot(PROJ.T, ZEIG) @ np.diag(aa) @ np.dot(ZEIG.T, PROJ) 
    )
    for aa, ZEIG, Z in zip(_eigvals, _eigvecs, ZS)
]
idx_sorts = [
    eig_argsort(eigdecomp[0], sort_by=EIGVAL_ORDER)[::-1] for eigdecomp in subspace_eigdecomps
]
subspace_eigdecomps = [
    (eigdecomp[0][idx_sort], eigdecomp[1][:, idx_sort])
    for eigdecomp, idx_sort in zip(subspace_eigdecomps, idx_sorts)
]
print(len(subspace_eigdecomps[0]))

cset_z_eigvals = [eigdecomp[0] for eigdecomp in subspace_eigdecomps]
cset_z_eigvecs = [eigdecomp[1] for eigdecomp in subspace_eigdecomps]

cset_eigvals = cset_z_eigvals
cset_eigvecs = cset_z_eigvecs
cset_eigvecs[0].shape

In [ ]:
N_MODE = 5

HY = 0.5*FIG_LY_MAX
if N_MODE > 3:
    HY = FIG_LY_MAX

fig = plt.figure(figsize=(FIG_LX_WIDE, HY), constrained_layout=True)
gs = mpl.gridspec.GridSpec(
    1+N_MODE, ncase, 
    figure=fig, height_ratios=[0.05*N_MODE]+[1]*N_MODE
)

axs = np.array([
    fig.add_subplot(gs[i, j]) 
    for i in range(1, gs.nrows)
    for j in range(gs.ncols)
]).reshape(gs.nrows-1, gs.ncols)
ax_cbar = fig.add_subplot(gs[0, :])
# axs = np.atleast_1d(axs)

coords = MESH.coordinates()
cells = MESH.cells()

if FUNCTIONAL_TYPE == 'OnsetPressure':
    symb = r"${d^2 p_\mathrm{on}} / {d E^2}$"
elif FUNCTIONAL_TYPE == 'OnsetFrequency':
    symb = r"${d^2 f_\mathrm{on}} / {d E^2}$"
else:
    raise ValueError(f"Unknown `FUNCTIONAL_TYPE` {FUNCTIONAL_TYPE}")

# dparams = d2fon_dparams2
# eigvals = eigval_d2fon_dparams2
        
## Plot emod sensitivity
zs = [
    cset_eigvecs[irow][:, jcol] 
    for jcol, irow in itertools.product(range(N_MODE), range(ncase))
]
zs_norm = [
    z/(z.max()-z.min()) for z in zs
]
eigvals = [
    cset_eigvals[irow][jcol] 
    for jcol, irow in itertools.product(range(N_MODE), range(ncase))
]
if FUNCTIONAL_TYPE == 'OnsetPressure':
    # kwargs = {'vmin': -0.01, 'vmax': 0.01}
    kwargs = {}
elif FUNCTIONAL_TYPE == 'OnsetFrequency':
    kwargs = {}
    
artists = plot_triplots(
    fig, axs.flat, len(zs)*[coords], len(zs)*[cells], zs_norm, lw=0, **kwargs
)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(f"Eigenvectors of {symb} [kPa]")

## Plot tmesh sensitivity
#     xy_med = coords[VERTS_MED]

#     med_dofs = np.array(
#         VEC_CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_MED)
#     ).reshape(-1, 2)
#     dir_dofs = np.array(
#         CG1_DOFMAP.entity_closure_dofs(MESH, 0, VERTS_DIR)
#     ).reshape(-1, 2)
#     for ax, dparams in zip(axs[ii, :], dparams_list):
#         tmesh = dparams['tmesh']
#         tmesh_med = tmesh[med_dofs]
#         ax.quiver(*xy_med.T, *tmesh_med.T, label="Shape traction")

for ax in axs.flat:
    ax.set_aspect(1)
    ax.set_ylim(0, 0.6)
    
for ax, eigval in zip(axs.flat, eigvals):
    ax.text(
        0.05, 0.95, r"$\lambda=$ " + f"{eigval:.1f}", 
        transform=ax.transAxes, va='top'
    )

case_labels = [
    f"({emod_cov/10/1e3:.1f} kPa, {emod_bod/10/1e3:.1f} kPa)" 
    for emod_cov, emod_bod in zip(emods_cov, emods_bod)
]
case_labels[0] = "$(E_{\mathrm{b}}, E_{\mathrm{c}})$ = \n" + case_labels[0]
for ax, case_label in zip(axs[0, :], case_labels):
    ax.text(0.05, 1.05, case_label, transform=ax.transAxes)

for ax in axs[-1, :]:
    ax.set_xlabel("x [cm]")
for ax in axs[:, 0]:
    ax.set_ylabel("y [cm]")

for ax in axs[:, 1:].flat:
    ax.tick_params('y', labelleft=False)
    
for ax in axs[:-1, :].flat:
    ax.tick_params('x', labelbottom=False)

# fig.tight_layout()

fig.savefig(f'{FIG_DIR}/Hessian_{FUNCTIONAL_TYPE}_{BASIS_TYPE}_{EIGVAL_ORDER}_{N_MODE}.{FIG_EXT}')

#### Linear stability modes vs linearization point

In [ ]:
def _control(xhopf):
    HOPF_MODEL.set_state(xhopf)
    return HOPF_MODEL.res.control.copy()
    
states_fp = [xhopf[HOPF_MODEL.labels_fp] for xhopf in cset_xhopf]
controls_fp = [_control(xhopf) for xhopf in cset_xhopf]
eigmodes = [
    libhopf.solve_linear_stability(HOPF_MODEL.res, state_fp, control, prop)[0]
    for state_fp, control, prop in tqdm(zip(states_fp, controls_fp, cset_props))
]

In [ ]:
ponsets = np.array([x.sub['psub'][0] for x in cset_xhopf])
fonsets = np.array([x.sub['omega'][0]/2/np.pi for x in cset_xhopf])

In [ ]:
NUM_MODE = 5
eigmodes_ = np.array([x[:2*NUM_MODE:2] for x in eigmodes])

In [ ]:
fig = plt.figure(figsize=(FIG_LX, FIG_LY)) 
gs = gridspec.GridSpec(2, 1, figure=fig)
axs = np.array([fig.add_subplot(gs[i, 0]) for i in range(gs.nrows)])

axs[0].plot(
    np.arange(eigmodes_.shape[0])+1, ponsets/10,
    label='$p_\\mathrm{on}$'
)
ax_twin = axs[0].twinx()
ax_twin.plot(
    np.arange(eigmodes_.shape[0])+1, fonsets,
    label='$f_\\mathrm{on}$'
)

for nmode in range(eigmodes_.shape[1]):
    axs[1].scatter(
        np.arange(eigmodes_.shape[0])+1, eigmodes_[:, nmode].imag/2/np.pi, 
        label=f"Mode {nmode+1:d}"
    )
    
ax.set_xlabel("Linearization point")
ax.set_ylabel("Mode frequency [Hz]")
ax.legend()

### Sensitivity results around a single base point

#### Define the base point

In [ ]:
emod = 6.0e4
params_pon = {
    'Ecov': emod, 'Ebod': emod, 
    'Functional': 'OnsetPressure', 
    'ParamOption': 'const_shape'
}

params_fon = {
    'Ecov': emod, 'Ebod': emod, 
    'Functional': 'OnsetFrequency', 
    'ParamOption': 'const_shape'
}

prop_base = load_sensitivity_vectors(params_pon, 'prop')[0]
PARAMETERIZATION, scale = setup_parameterization(params_pon, HOPF_MODEL, prop_base)
param_base = load_sensitivity_vectors(params_pon, 'param')[0]

E0 = prop_base.sub['emod']

xhopf_base = load_sensitivity_vectors(params_pon, 'state')[0]

eigvals = load_sensitivity_scalars(params_pon, 'eigvals')

eigvecs = [
    evec.sub['emod']
    for evec in load_sensitivity_vectors(params_pon, 'hess_param')
]
print(eigvals)

grad_f = load_sensitivity_vectors(params_pon, 'grad_param')[0].sub['emod']
grad_c = load_sensitivity_vectors(params_fon, 'grad_param')[0].sub['emod']

#### Quadratic model minimization

In [ ]:
## Solve the quadratic minimization problem
# min 1/2 x^T d2f_dx2 x + df_dx x
# 'a' will represent the vector of coefficient in the reduced basis

use_frequency_constraint = False

Z_EIG = form_basis(eigvecs)
LAMBDA_EIG = form_principal_sensitivity(eigvals[:])

# 'Z' is the reduced dimensional basis to solve for changes in E, in
# NOTE: For `Z_BC_CINFSUP_BINFSUP`, using the uniform basis component results in very negative 
# predictions for the minimum onset pressure
Z = Z_EIG[:, :2]
Z = Z_BC_CINFSUP_BINFSUP[:, 1:2]

d2f_da2 = (Z.T @ Z_EIG) @ LAMBDA_EIG @ (Z_EIG.T @ Z)
df_da = grad_f @ Z
dc_da = grad_c @ Z

print(d2f_da2, df_da)
print(np.linalg.solve(d2f_da2, -df_da))

# 'x' represents the block form [modulus change, -frequency lagrange multiplier]
x0 = np.zeros(3)

if use_frequency_constraint:   
    A = np.block(
        [[             d2f_da2, dc_da.reshape(-1, 1)],
         [dc_da.reshape(1, -1),     np.zeros((1, 1))]]
    )
else:
    dc_da = 0*dc_da
    A = np.block(
        [[             d2f_da2, dc_da.reshape(-1, 1)],
         [dc_da.reshape(1, -1),     np.ones((1, 1))]]
    )
    
b = np.block(
    [df_da-x0[-1]*dc_da, np.zeros((1, 1))]
)

x = np.linalg.solve(A, -b.reshape(-1))

In [ ]:
DE_min = Z @ x[:-1]

print(x)
print(np.linalg.norm(DE_min))

In [ ]:
# Use the parameterization to map the change in parameters to a change in base properties
delta_param_min = PARAMETERIZATION.x.copy()
delta_param_min[:] = 0
delta_param_min['emod'] = DE_min
delta_props_min = PARAMETERIZATION.apply_jvp(param_base, delta_param_min)

In [ ]:
## Compute onset pressure/frequency at the starting/minimum point
props_min = prop_base + delta_props_min

# xhopf_min_n = xhopf
xhopf_min, info = libhopf.solve_hopf_by_newton(HOPF_MODEL, xhopf_base, prop_base+delta_props_min)
print(info)
# alphas = np.linspace(0, 1, 21)
# for alpha in tqdm(alphas):
#     xhopf_min_n, info = libhopf.solve_hopf_by_newton(HOPF_MODEL, xhopf_min_n, prop+alpha*delta_props_min)
#     print(info)

fonsets = 1/(2*np.pi)*np.array(
    [x.sub['omega'][0] for x in (xhopf_base, xhopf_min)]
)
ponsets = 1/10*np.array(
    [x.sub['psub'][0] for x in (xhopf_base, xhopf_min)]
)

print(ponsets)

In [ ]:
fig = plt.figure(figsize=(FIG_LX_WIDE, 10*INCH))
grid_spec = mpl.gridspec.GridSpec(2, 2, figure=fig, height_ratios=[0.025, 1])
ax_cbar = fig.add_subplot(grid_spec[0, :])
axs = np.array([fig.add_subplot(grid_spec[1, n]) for n in range(grid_spec.ncols)])

zs = [x.sub['emod']/10/1e3 for x in (prop_base, props_min)]
coords = MESH.coordinates()
cells = MESH.cells()
artists = plot_triplots(fig, axs, len(zs)*[coords], cells, zs, lw=0)

fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, fonset, ponset in zip(axs.flat, fonsets, ponsets):
    text = "$(f, p)_\mathrm{onset}=$" + f"({fonset:.1f} Hz, {ponset/1:.1f} Pa)"
    ax.text(0, 1, text, transform=ax.transAxes, ha='left', va='top')

ax_cbar.set_xlabel("$E$ [kPa]")
ax_cbar.xaxis.set_label_position('top')
ax_cbar.xaxis.set_tick_params(
    labelbottom=False, labeltop=True,
    bottom=False, top=True
)

axs[1].yaxis.set_tick_params(labelleft=False)

labels = ["Original distribution", "Onset pressure minimizer"]
for ax, label in zip(axs, labels):
    ax.text(0, 1, label, transform=ax.transAxes, ha='left', va='bottom')

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
    
axs[0].set_ylabel("y [cm]")

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/QuadraticApproximateMinimizer_{BASIS_TYPE}.{FIG_EXT}')

#### Structural eigenmodes along principal sensitivity

In [ ]:
# Generate a list of properties along a perturbation direction
alphas = np.linspace(-10, 10, 6)
dparam = PARAMETERIZATION.x.copy()
dparam[:] = 0
dparam['emod'] = Z_EIG[:, 0]

params = [param_base + alpha*dparam for alpha in alphas]
props = [PARAMETERIZATION.apply(_param) for _param in params]

In [ ]:
[prop['emod'][:].max() for prop in props]

In [ ]:
def modal_decomp(model, prop):
    """
    Return the modal decomposition of the model
    """
    state_fp_ini = model.state.copy()
    state_fp_ini[:] = 0
    control = model.control
    control[:] = 0
    control['psub'] = 1e-5
    state_fp, info = libhopf.solve_fp(model, control, prop, state_fp_ini)
    # print(info)
    
    evals, evecs_r, evecs_i = libhopf.solve_linear_stability(model, state_fp, control, prop)
    return evals, evecs_r, evecs_i

In [ ]:
evals_trend = []
nmode = 5
for prop in props:
    evals, evecs_r, evecs_i = modal_decomp(HOPF_MODEL.res, prop)
    # print(evals)
    evals_trend.append(evals[:nmode*2:2])
    
# The shape is (# parameter points, # modes)
evals_trend = np.array(evals_trend)

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(FIG_LX, FIG_LY)
)

for n in range(nmode):
    ax.plot(
        alphas, evals_trend[:, n].imag/2/np.pi, 
        label=f"Mode {n:d}"
    )
    
ax.legend()
ax.set_ylabel("Frequency [Hz]")
ax.set_xlabel("$\\alpha$")

fig.tight_layout()
fig.savefig(f'{FIG_DIR}/StructuralModesAlong2ndOrderSensitivity.{FIG_EXT}')

In [ ]:
# Plot modes at the base point
AMP = 500
N_PHASE = 5
N_MODE = 4
PHASES = np.linspace(0, 1, N_PHASE+1)[:N_PHASE]

VERTS_MED = SDOF_TO_VERT[HOPF_MODEL.res.fsimap.dofs_solid]
# VERTS_MED = 
# TSHAPE_PARAM = 

fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY))
gs = gridspec.GridSpec(N_MODE, N_PHASE, figure=fig)
axs = np.array(
    [[fig.add_subplot(gs[i, j]) for j in range(gs.ncols)] for i in range(gs.nrows)]
)

evals, evecs_r, evecs_i = modal_decomp(HOPF_MODEL.res, prop_base)
for nmode in range(N_MODE):
    print(evals[2*nmode])
    er = evecs_r[2*nmode]['u'][:]
    ei = evecs_i[2*nmode]['u'][:]
    
    for ax, phase in zip(axs[nmode, :], PHASES):
        # print(ax, phase)
        dcoords = AMP*np.real(
            (er + 1j*ei)*np.exp(phase*2*np.pi*1j)
        )
        dcoords = dcoords[VERT_TO_VDOF].reshape(-1, 2)
        coords_disp = coords+dcoords
        ax.triplot(*coords_disp.T, cells, lw=0.5)
        
        ax.plot(*coords[VERTS_MED].T, color='k', lw=0.5)

for ax in axs.flat:
    ax.set_adjustable('box')
    ax.set_aspect(1)
    ax.set_xlim(-0.2, 0.9)
    ax.set_ylim(0, 0.6)
    
for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)
    
fig.savefig(f'{FIG_DIR}/StructuralModes.{FIG_EXT}')

#### Compare nonlinear onset pressure variation and taylor sensitivity model

In [ ]:
ZEIG = form_basis(eigvecs)

# Find eigendecomposition in a given basis
Z = form_basis(eigvecs[:2])
# Z = Z_BC_CINFSUP_BINFSUP
r_eigvals, r_eigvecs = np.linalg.eigh((Z.T@ZEIG) @ np.diag(eigvals) @ (ZEIG.T@Z))
idx_sort = eig_argsort(r_eigvals, sort_by='real')[::-1]
r_eigvals, r_eigvecs = r_eigvals[idx_sort], r_eigvecs[idx_sort]

print(r_eigvals)
print(r_eigvecs)

In [ ]:
## Sanity check that minimum corresponds to large deviation
dr = np.linalg.solve(np.diag(r_eigvals), -r_eigvecs.T @ (grad_f @ Z))

da = r_eigvecs@dr
print(da)

In [ ]:
# np.linalg.norm(r_eigvecs[:, 1])

In [ ]:
xs = np.linspace(-5, 5, 6)
xs = np.linspace(-1, 1, 11)

dirs = []

nmode = 1
for n in tqdm(range(nmode)):
    dparam = PARAMETERIZATION.x.copy()
    dparam[:] = 0
    dparam['emod'] = Z@r_eigvecs[:, n]
    
    def _solve(model, xhopf, param, dparam):
        dprop = PARAMETERIZATION.apply_jvp(param, dparam)
        
        xhopf_n, info = libhopf.solve_hopf_by_newton(
            HOPF_MODEL, xhopf, prop_base+dprop
        )
        if info['status'] != 0:
            print("didn't converge!")
        return xhopf_n
    
    ponsets = [
        _solve(HOPF_MODEL, xhopf_base, param_base, x*dparam).sub['psub'][0]
        for x in tqdm(xs)
    ]
    
    dirs.append(np.array(ponsets))

In [ ]:
fig, ax = plt.subplots(1, 1)

colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']
for n, (eigval, r_eigvec, ponsets) in enumerate(zip(r_eigvals, r_eigvecs.T, dirs)):
    color = colors[n]
    
    con = xhopf_base.sub['psub'][0]
    lin = (grad_f).dot(Z@r_eigvec)
    quad = eigval
    print(eigval)
    
    ax.plot(xs, np.array(ponsets)/10, color=color, label='Exact')
    ax.plot(
        xs, (con + lin*xs + 1/2*quad*xs**2)/10, 
        color=color, ls='-.', label='Taylor approximation'
    )

ax.set_xlabel("$a$")
ax.set_ylabel("$p_\mathrm{onset}$ [Pa]")
ax.legend()

#### Compare sensitivity across different subspaces

In [ ]:
Z_EIG = form_basis(eigvecs)

In [ ]:
# Form eigendecompositions of hessian sensitivity for each subspace of interest
basis_names = [
    'unstructured',
    'body-cover \nw/cover and \nbody gradient', 
    'body-cover \nw/cover gradient', 
    'body-cover'
]

# Bases spanning each subspace
bases = [
    Z_EIG, 
    Z_BC_CINFSUP_BINFSUP,
    Z_BC_CINFSUP,
    Z_BC
]

projectors = [
    form_projector(Z) for Z in bases
]

subspace_eigdecomps = [
    np.linalg.eigh( 
        np.dot(proj.T, Z_EIG) @ np.diag(eigvals) @ np.dot(Z_EIG.T, proj) 
    )
    for proj in projectors
]
idx_sorts = [
    eig_argsort(eigdecomp[0], sort_by='real')[::-1] 
    for eigdecomp in subspace_eigdecomps
]
subspace_eigdecomps = [
    (eigdecomp[0][idx_sort], eigdecomp[1][:, idx_sort])
    for eigdecomp, idx_sort in zip(subspace_eigdecomps, idx_sorts)
]

cset_grads = [proj@grad_f for proj in projectors]
# cset_grads = [grad_f@proj for proj in projectors]

cset_eigvals = [eigdecomp[0] for eigdecomp in subspace_eigdecomps]
cset_eigvecs = [eigdecomp[1] for eigdecomp in subspace_eigdecomps]
# cset_eigvals[0]

##### Compare gradient and hessian in restricted subspaces

In [ ]:
## Plot gradient

fig = plt.figure(
    figsize=(FIG_LX_WIDE, 0.8*FIG_LY), constrained_layout=True
)
gs = gridspec.GridSpec(
    2, len(cset_eigvecs), height_ratios=[0.05, 1], figure=fig
)

axs = np.array([fig.add_subplot(gs[1, n]) for n in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, :])

zs = cset_grads
zs_vert = [project_DG0_to_CG1(z)[VERT_TO_SDOF] for z in zs]
lmbdas = [eigvals[0] for eigvals in cset_eigvals]
artists = plot_triplots(fig, axs, [coords]*len(zs), cells, zs)
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontour', levels=[0], colors='k')
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontourf')
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs, basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)
    ax.set_aspect(1)
    
for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax in axs:
    ax.set_xlabel("x [cm]")
    
axs[0].set_ylabel("y [cm]")
    
ax_cbar.set_xlabel("${{dp_\\mathrm{on}}} / {{dE}}$ [kPa]")
# fig.tight_layout()

fig.savefig(f"{FIG_DIR}/GradientComparisonAcrossSubspaces.{FIG_EXT}")

In [ ]:
## Plot gradient without principal sensitivity component

fig = plt.figure(figsize=(FIG_LX_WIDE, 0.8*FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(2, len(cset_eigvecs), height_ratios=[0.05, 1], figure=fig)

axs = np.array([fig.add_subplot(gs[1, n]) for n in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, :])

# print([np.inner(eigvecs[:, 0], eigvecs[:, 0]) for eigvecs in cset_eigvecs])
# print([
#     np.dot(grad, eigvecs[:, 0])
#     for grad, eigvecs in zip(cset_grads, cset_eigvecs)
# ])
zs = [
    grad 
    - np.inner(grad, eigvecs[:, 0])*eigvecs[:, 0]
    # - np.inner(grad, eigvecs[:, 1])*eigvecs[:, 1]
    for grad, eigvecs in zip(cset_grads, cset_eigvecs)
]

print([np.linalg.norm(z) for z in zs])

lmbdas = [eigvals[0] for eigvals in cset_eigvals]
artists = plot_triplots(fig, axs, [coords]*len(zs), cells, zs)
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontour', levels=[0], colors='k')
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs_vert, plot_type='tricontourf')
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs, basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)
    ax.set_aspect(1)
    
for ax in axs[1:]:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax in axs:
    ax.set_xlabel("x [cm]")
    
axs[0].set_ylabel("y [cm]")
    
ax_cbar.set_xlabel("${{dp_\\mathrm{on}}} / {{dE}} (I-H_1 H_1^{T})$ [kPa]")
# fig.tight_layout()

fig.savefig(f"{FIG_DIR}/GradientWOHessianComparisonAcrossSubspaces.{FIG_EXT}")

In [ ]:
N_MODE = 2
fig = plt.figure(figsize=(FIG_LX_WIDE, FIG_LY), constrained_layout=True)
gs = gridspec.GridSpec(
    1+N_MODE, len(cset_eigvecs), height_ratios=[0.05]+[1]*N_MODE,
    figure=fig
)

axs = np.array([
    [fig.add_subplot(gs[i, n]) for n in range(gs.ncols)] 
     for i in range(1, N_MODE+1)
])
ax_cbar = fig.add_subplot(gs[0, :])

for nmode in range(N_MODE):
    zs = [eigvecs[:, nmode] for eigvecs in cset_eigvecs]
    zs_norm = [z/(z.max()-z.min()) for z in zs]
    lmbdas = [eigvals[nmode] for eigvals in cset_eigvals]
    artists = plot_triplots(fig, axs[nmode, :], [coords]*len(zs), cells, zs_norm)
    
    for ax, lmbda in zip(axs[nmode, :].flat, lmbdas):
        text = f"$\\lambda = {lmbda:.1f}$"
        ax.annotate(text, (0.05, 0.95), va='top', xycoords=ax.transAxes)
    
# plot_triplots(fig, axs, [coords]*len(zs), cells, zs, plot_type='tricontourf', levels=[0])
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')

for ax, basis_name, lmbda in zip(axs[0, :], basis_names, lmbdas):
    text = f"{basis_name}"
    ax.annotate(text, (0, 1.05), xycoords=ax.transAxes)
    
for ax in axs.flat:
    ax.set_aspect(1)
    
for ax in axs[:, 1:].flat:
    ax.yaxis.set_tick_params(labelleft=False)
    
for ax in axs[:-1, :].flat:
    ax.xaxis.set_tick_params(labelbottom=False)
    
for ax in axs[-1, :].flat:
    ax.set_xlabel("x [cm]")
    
for ax in axs[:, 0].flat:
    ax.set_ylabel("y [cm]")
    
ax_cbar.set_xlabel("${{d^2p_\\mathrm{on}}} / {{dE^2}}$ [kPa]")
# fig.tight_layout()

fig.savefig(f"{FIG_DIR}/EigenmodeComparisonAcrossSubspaces.{FIG_EXT}")

##### Compare theoretical minimum onset pressure

In [ ]:
# Set the one-dimensional approximation to solve for the minimum along
es = [eigvecs[:, 0] for eigvecs in cset_eigvecs]
# print([np.linalg.norm(e) for e in es])
lmbdas = [eigvals[0] for eigvals in cset_eigvals]
# print(lmbdas)

grads_alpha = [grad_f.dot(e) for e in es]
hesss_alpha = [lmbda for lmbda in lmbdas]
alphas = [
    -grad_alpha/hess_alpha 
    for grad_alpha, hess_alpha in zip(grads_alpha, hesss_alpha)
]

pon_base = xhopf_base.sub['psub'][0]
dpon_alphas = [
    0.5*hess_alpha*alpha**2+grad_alpha*alpha 
    for alpha, grad_alpha, hess_alpha in zip(alphas, grads_alpha, hesss_alpha)
]
pons = [pon_base+dpon for dpon in dpon_alphas]
print(np.array(pons)/10)
print(np.array(dpon_alphas)/10)
print(alphas)

##### Compare nonlinear onset pressure variation

In [ ]:
# Directions of principal sensitivities to compare
dirs = [
    eigvecs[:, 0] for eigvecs in cset_eigvecs 
]

alphas = np.linspace(-5, 30, 11)
_alphas = np.linspace(-5, 10, 11)

pon_base = xhopf_base.sub['psub'][0]
grads_alpha = [grad_f.dot(e) for e in dirs]
hesss_alpha = [eigvals[0] for eigvals in cset_eigvals]

In [ ]:
ponsets_vs_dirs = []
for z in tqdm(dirs):
    dparam = PARAMETERIZATION.x.copy()
    dparam[:] = 0
    dparam['emod'][:] = z
    params = [param_base + alpha*dparam for alpha in alphas]
    prop = [PARAMETERIZATION.apply(_param) for _param in params]
    
    def _solve_hopf_by_newton(hopf, xhopf_0, prop, **kwargs):
        xhopf, info = libhopf.solve_hopf_by_newton(
            hopf, xhopf_0, prop, **kwargs
        )
        if info['status'] != 0:
            print("Newton solver for Hopf system did not converge")
        return xhopf
    
    ponsets = np.array([
        _solve_hopf_by_newton(HOPF_MODEL, xhopf_base, prop).sub['psub'][0]
        for prop in tqdm(prop)
    ])
    
    ponsets_vs_dirs.append(ponsets)

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(FIG_LX_WIDE, FIG_LY)
)

for color, basis_name, direction, ponsets in zip(COLOR_CYCLE, basis_names, dirs, ponsets_vs_dirs):
    ax.plot(alphas, ponsets/10, label=basis_name, color=color)
    
for color, basis_name, grad_alpha, hess_alpha in zip(COLOR_CYCLE, basis_names, grads_alpha, hesss_alpha):
    ax.plot(
        _alphas, (pon_base + grad_alpha*_alphas + 1/2*hess_alpha*_alphas**2)/10, 
        ls='-.', color=color, alpha=0.5, lw=5
    )
    
ax.set_ylabel("$p_\\mathrm{{on}}$ [Pa]")
ax.set_xlabel("$\\alpha$ []")
ax.legend()

fig.savefig(f'{FIG_DIR}/OnsetPressureVariationComparisonBetweenSubspacesAlong2ndOrderSensitivity.{FIG_EXT}') 

### Nonlinear minimization Results


In [ ]:
emods = EMODS

# functional_name = 'OnsetFrequency'
functional_name = 'OnsetPressure'
# functional_name = 'OnsetPressureStrainEnergy'

param_option = 'const_shape'
# param_option = 'all'

it = -1
hopf_states = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'hopf_state', it
    )[0] 
    for emod in emods
]

cset_props = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'hopf_props', it
    )[0] 
    for emod in emods
]

params_list = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'parameters', it
    )[0] 
    for emod in emods
]

dcset_props = [
    load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'grad', it
    )[0] 
    for emod in emods
]

omegas = [
    np.abs(load_minimization_vectors(
        {'Ecov': emod, 'Ebod': emod, 'ParamOption': param_option, 'Functional/Name': functional_name}, 
        'hopf_state', 0
    )[0]['omega'][0])
    for emod in emods
]

emod = cset_props[0].sub['emod']
emod.min() - emod.max()

In [ ]:
fig = plt.figure(figsize=(15, 5))
gs = mpl.gridspec.GridSpec(2, len(emods), figure=fig, height_ratios=[0.05, 0.95])
axs = np.array([fig.add_subplot(gs[-1, i]) for i in range(gs.ncols)])
ax_cbar = fig.add_subplot(gs[0, 0:3])

def get_coords(prop, state):
    HOPF_MODEL.set_prop(prop)
    ufp = state['u']
    ufp_vert = np.array(ufp[VERT_TO_VDOF]).reshape(-1, 2)
    
    coords = HOPF_MODEL.res.solid.forms['mesh.mesh'].coordinates() + ufp_vert
    return coords.copy()

coordss = [get_coords(prop, state) for prop, state in zip(cset_props, hopf_states)]
coords = coordss[-1]
cells = MESH.cells()
print([np.linalg.norm(coords) for coords in coordss])

_fspace_cg1 = HOPF_MODEL.res.solid.forms['fspace.scalar']
_fspace_dg0 = HOPF_MODEL.res.solid.forms['fspace.scalar_dg0']
_function_dg0 = dfn.Function(_fspace_dg0)
_function_cg1 = dfn.Function(_fspace_cg1)
def project_dg0_to_cg1(vec):
    _function_dg0.vector()[:] = vec
    # _function_cg1.vector()[:] = vec
    dfn.project(_function_dg0, V=_fspace_cg1, function=_function_cg1)
    return np.array(_function_cg1.vector()[:])

VERT_TO_SDOF = dfn.vertex_to_dof_map(_fspace_cg1)
zs = [
    project_dg0_to_cg1(prop['emod'])[VERT_TO_SDOF]/10/1e3 
    for prop in cset_props
]
# print(zs[0])
artists = plot_triplots(
    fig, axs, coordss, cells, zs, 
    plot_type='tricontourf', extend='neither', levels=np.linspace(0, 30, 11), lw=0
)
# print(artists[0])
# artists[0] + artists[1]
fig.colorbar(artists[0], cax=ax_cbar, orientation='horizontal')
ax_cbar.set_xlabel(r"$E$ [kPa]")
# ax_cbar.set_xlabel(r"$\frac{dg}{d\mathbf{E}}$")

for ax in axs:
    ax.set_aspect(1)
    ax.set_xlabel("x [cm]")
    ax.set_ylim(0, 0.52)
    ax.set_xlim(0, 0.8)
    
for ax, omega in zip(axs, omegas):
    ax.text(0, 1.05, f"$f_0$={omega/2/np.pi:.1f} Hz", transform=ax.transAxes)
axs.flat[0].set_ylabel("y [cm]")

for ax in axs[1:]:
    ax.tick_params('y', labelleft=False)

fig.tight_layout()

In [ ]:
## Global vars
ZETA = 1e-4
R_SEP = 1.0
Y_GAP = 1e-2
# PSUBS = np.concatenate([np.arange(200, 300, 10), np.arange(300, 1000, 100)])*10
PSUBS = np.arange(550, 650, 10) * 10 
ECOV = 5e3 * 10
# EBODY = 15e3 * 10
EBODY = 5e3 * 10

# Load the transient model
MESH_NAME = 'BC-dcov5.00e-02-cl1.00'
MESH_NAME = 'M5_CB_GA3'

mesh_path = f'mesh/{MESH_NAME}.msh'

kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}
model_tran = load_tran(mesh_path, **kwargs)

# load the Hopf models
res_hopf, res, dres = load_hopf(mesh_path, **kwargs)

# print(model_tran.solid.forms['mesh.vertex_label_to_id'])
# print(model_tran.solid.forms['mesh.facet_label_to_id'])
# print(model_tran.solid.forms['mesh.cell_label_to_id'])

In [ ]:
## Initialize functions to extract glottal width from saved simulations

# Number of points per period to use in processing
N_PER_PERIOD = 100
proc_gw_tran = make_sig_glottal_width_sharp(model_tran)
def time(f):
    return np.array(f.get_times())

In [ ]:
## Load mesh and DOF map information for plotting data on the mesh
TRI = res.solid.forms['mesh.mesh'].coordinates()
X, Y = TRI[:, 0], TRI[:, 1]
CELLS = res.solid.forms['mesh.mesh'].cells()

# These indices select DOFs from a scalar function in vertex-order
IDX_VERT = dfn.vertex_to_dof_map(res.solid.forms['fspace.scalar'])

XREF = res.solid.XREF.vector()
VERT_TO_VDOF = dfn.vertex_to_dof_map(res.solid.forms['fspace.vector'])

# Conversion for centimeter
CM = 1/2.54

## Miscellaneous experiments

### Growth rate plot for generic model

Plots of the growth rate vs subglottal pressure.

In [ ]:
MESH_NAME = 'M5_CB_GA1'

mesh_path = f'mesh/{MESH_NAME}.msh'

kwargs = {
    'sep_method': 'fixed',
    'sep_vert_label': 'separation-inf'
}
model_tran = load_tran(mesh_path, **kwargs)

# load the Hopf models
res_hopf, res, dres = load_hopf(mesh_path, **kwargs)

In [ ]:
for key, subvec in res.prop.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")
    
for key, subvec in res.control.items():
    print(f"({key}) mean, min, max: {np.mean(subvec)}, {np.min(subvec.array)}, {np.max(subvec.array)}")

In [ ]:
psubs = np.arange(0, 800, 20)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 5), sharex=True)

axs[0].plot(psubs, modes.real)
axs[1].plot(psubs, modes.imag)

axs[0].axhline(0, color='k', ls='-.')

axs[0].set_ylabel("$\omega_{real}$")
axs[1].set_ylabel("$\omega_{imag}$")
axs[1].set_xlabel("$p_{sub}$")

# axs[0].legend()
# axs[1].legend()

fig.tight_layout()

### Bifurcation plots for smooth minimum separation fluid models

In [ ]:
# Set the model properties and sanity check the values
_region_to_dofs = meshutils.process_celllabel_to_dofs_from_forms(
    res.solid.forms, res.solid.forms['fspace.scalar']
)
prop = libsetup.set_prop(res.prop, _region_to_dofs, res)

prop['zeta_min'] = 1e-8
prop['zeta_sep'] = 1e-4
res.set_prop(prop)

# proplabel_to_norm = {label: subvec.norm() for label, subvec in prop.items()}
# pprint(proplabel_to_norm)

proplabel_to_max = {label: subvec.max() for label, subvec in prop.items()}
pprint(proplabel_to_max)
print(res.solid.forms['mesh.mesh'].coordinates()[..., 1].max())

#### Plot the growth rate vs $p_{sub}$

In [ ]:
zeta_min = 1e-6
zeta_sep = 1e-2
prop = res.prop
prop['zeta_min'] = zeta_min
prop['zeta_sep'] = zeta_sep
res.set_prop(prop)

# psubs = np.arange(585, 590, 0.1)*10
psubs = np.arange(700, 800, 10)*10
# psubs = np.arange(700, 800, 2)*10
modes = [libhopf.solve_least_stable_mode(res, psub)[0] for psub in tqdm(psubs)]
modes = np.array(modes)

In [ ]:
def jac_block_norms(res, psub):
    xfp, info = libhopf.solve_fp(res, psub)
    res.set_state(xfp)
    res.control['psub'].array[0] = psub
    res.set_control(res.control)
    
    dres_dstate = res.assem_dres_dstate()
    return tuple([submat.norm() for submat in dres_dstate.subarrays_flat])

block_norms = [jac_block_norms(res, psub) for psub in tqdm(psubs)]
block_labels = [','.join(multi_label) for multi_label in itertools.product(res.state.labels[0], res.state.labels[0])]

block_norms = np.array(block_norms)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(5, 5), sharex=True)
    
ii_max = np.argmax(modes.real)
print(psubs[ii_max])

axs[0].plot(psubs, modes.real)
axs[0].axhline(0, color='k', ls='-.')
axs[0].set_ylabel("$\omega_{real}$")

axs[1].plot(psubs, modes.imag)
axs[1].set_ylabel("$\omega_{imag}$")

for norms, label in zip(block_norms.T, block_labels):
    axs[2].plot(psubs, norms, label=label)
axs[2].set_ylabel("$|| \\frac{dF}{dx} ||$")
axs[2].set_xlabel("$p_{sub}$")
# axs[2].set_yscale('log')
axs[2].legend()

# axs[0].set_ylim(-10, 50)

fig.savefig(f'fig/GrowthRatevsPsub_zetamin{zeta_min:.2e}_zetasep{zeta_sep:.2e}_closeup.png', dpi=200)

### Strange bifurcation behaviour for smoothed minimum separation models

Rapid changes in eigenvalues can occur with small changes in $p_{sub}$ when using a smooth minimum approximation. This happens when the smooth minimum approximation approaches the true minimum where small changes in areas rapidly shift weights between nodes.

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True)

s = np.linspace(0, 1, 12)
a = (s-0.5)**2 + 0.1
da = 1e-2*s

def smoothmin(a, zeta=1e-4):
    log_w = -a/zeta
    log_w = log_w - log_w.max()
    print(log_w)
    _w = np.exp(log_w)
    print(_w.max())
    return _w/np.sum(_w)

axs[0, 0].plot(s, a, marker='.')
axs[0, 0].set_title("a [cm]")
axs[1, 0].plot(s, smoothmin(a))
axs[1, 0].set_ylim(0, 1)

axs[0, 0].set_ylabel("Area [cm]")
axs[1, 0].set_ylabel("smoothmin weight")

axs[0, 1].plot(s, a+da, marker='.')
axs[0, 1].set_title("a + da [cm]")
axs[1, 1].plot(s, smoothmin(a+da))
axs[1, 1].set_ylim(0, 1)

# axs[0, 0].yaxis.
for ax_row in axs:
    for col in range(1, ax_row.size):
        ax_row[0].sharey(ax_row[col])
        ax_row[col].yaxis.set_tick_params(labelleft=False)

## Debugging

For the case where we minimize onset pressure while maintaining a constant onset frequency, the optimization encounters a problem. This is illustrated below for the history file 'OPT_DEBUG.h5' which started from a uniform initial guess of 5 kPa.

At some of the last iterations, optimization progress stalls and the optimization routine fails with a zero size step on a line search. A likely reason why this occurs can be seen by plotting the maximum real eigenvalue as a function of subglottal pressure. The minimal subglottal pressure of 0.38 kPa occurs due to a very sharp peak in the profile. It is likely that as parameters change in the line search, this small peak disappears so that onset jumps to the next onset pressure of around 0.5 kPa.

In [ ]:
# Load optimization history
opt_fname = 'OPT_DEBUG_1'
with h5py.File(f'out/{opt_fname}.h5', mode='r') as f:
    objs, grads, params, xhopfs = load_opt_hist(f)
    
its = np.arange(len(objs))
grads_norm = np.array([bvec.norm(grad) for grad in grads])
onset_ps = np.array([xhopf['psub'][0] for xhopf in xhopfs])
onset_fs = np.array([xhopf['omega'][0] for xhopf in xhopfs])

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True)

axs[0].plot(its, objs)
axs[0].set_ylabel("Objective")

axs[1].plot(its, onset_ps)
axs[1].set_ylabel("Onset pressure")

axs[2].plot(its, np.abs(onset_fs))
axs[2].set_ylabel("Onset freq.")

axs[2].set_xlabel("")

### Compute a line search along a given direction

In [ ]:
# compute functionals along the line from N -> N+1
N = 25
res_hopf.set_state(bvec.convert_subtype_to_petsc(xhopfs[N]))
res_hopf.set_prop(bvec.convert_subtype_to_petsc(params[N]))

xhopf_n, info = libhopf.solve_hopf_newton(res_hopf, res_hopf.state)
print(info)
print(xhopf_n['psub'][0])

### Compute the spectrum at a specific iteration

In [ ]:
# Specify the parameter set from the optimization to use
N = 26
res.set_prop(bvec.convert_subtype_to_petsc(params[N][:-2]))

In [ ]:
## Plot the spectrum at the onset pressure

def plot_spectrum(fig, ax, xhopf):
    res.set_prop(bvec.convert_subtype_to_petsc(params[N][:-2]))
    res.set_control(res.control)

    xfp_n = xhopf[res_hopf.labels_fp]
    psub = xhopf['psub'][0]
    omegas, eigvecs_real, eigvecs_imag = libhopf.solve_modal(res, xfp_n, psub)
    
    ax.scatter(omegas.real, omegas.imag)
    ax.axhline(0, color='k')
    ax.axvline(0, color='k')

    ax.set_xlabel("$\Re(\omega)$")
    ax.set_ylabel("$\Im(\omega)$")
    ax.set_title(f"$P_\mathrm{{onset}}$={onset_ps[N]/10:.1f} Pa")
    ax.set_xlim(-50, 10)
    ax.set_ylim(-1200, 1200)

    fig.tight_layout()
    return fig, ax

# for n in range(0, 27):
#     fig, ax = plt.subplots(1, 1)
#     fig, ax = plot_spectrum(fig, ax, n)
#     fig.savefig(f"fig/Spectrum{n}.png")
#     plt.close(fig)
    
fig, ax = plt.subplots(1, 1)
fig, ax = plot_spectrum(fig, ax, xhopfs[N])

In [ ]:
## Compute the most unstable eigenvalue over a range of subglottal pressures
# This gives a rough idea at which subglottal pressure(s) (if any) onset will occur

# psubs = np.arange(300, 600, 2.5)*10
psubs = np.arange(0, 2600, 100)*10
omegas_max = np.zeros(psubs.shape, dtype=complex)

least_stable_modes_info = [libhopf.solve_least_stable_mode(res_hopf.res, psub) for psub in psubs]
omegas_max = np.array([least_stable_info[0] for least_stable_info in least_stable_modes_info])

# xfp_0 = res.state.copy()
# xfp_0.set(0.0)

# for ii, psub in enumerate(psubs):
#     res.control['psub'][0] = psub
#     res.set_control(res.control)

#     xfp_n, info = libhopf.solve_fixed_point_newton(res, xfp_0, newton_params={'max_iter': 20})
#     # print(f"Solving for fixed point took {info['num_iter']} iterations. Abs err {info['abs_errs'][-1]}")
#     omegas, eigvecs_real, eigvecs_imag = libhopf.solve_linear_stability(res, xfp_n)
    
#     idx_max = np.argmax(omegas.real)
#     omegas_max[ii] = omegas[idx_max]

In [ ]:
print(np.array(omegas_max).shape)
print(psubs.shape)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 4), sharex=True)

# idx = np.logical_and(psubs >= 2000, psubs <= 6000)
idx = np.ones(psubs.shape, dtype=bool)

axs[0].plot(psubs[idx], np.array(omegas_max)[idx].real)
axs[0].axhline(0, color='k')
axs[0].set_ylabel("$real \; \omega$ $[rad/s]$")


axs[1].plot(psubs[idx], np.array(omegas_max)[idx].imag)
axs[1].axhline(0, color='k')
axs[1].set_ylabel("$imag \; \omega$ $[rad/s]$")

# axs[1].set_xlim(3500, 4000)5
# axs[0].set_ylim()


axs[1].set_xlabel("$P_{sub}$ $[Pa]$")